### Compute desired orderings

In [618]:
import random
random.seed(1)
import sys
sys.path.append("../../src/")
import uncertainpy.gradual as grad
import numpy as np
from tqdm import tqdm
import time
from scipy.stats import kendalltau, spearmanr

In [619]:
# get args in a layer
def get_layer_nodes(layer_sizes, layer_index=None):
    node_id = 0
    for i, size in enumerate(layer_sizes):
        if i == layer_index:
            return [str(n) for n in range(node_id, node_id + size)]
        node_id += size

    raise ValueError(f"Invalid layer_index {layer_index}: must be between 0 and {len(layer_sizes) - 1}")

In [620]:
# obtain input_args, hidden_args, output_args
def get_layer_args(layer_sizes):
    input_size = layer_sizes[0]
    output_size = layer_sizes[-1]
    total_neurons = sum(layer_sizes)

    input_args = [str(i) for i in range(input_size)]

    hidden_start = input_size
    hidden_end = total_neurons - output_size
    hidden_args = [str(i) for i in range(hidden_start, hidden_end)]

    output_args = [str(i) for i in range(hidden_end, total_neurons)]

    return input_args, hidden_args, output_args

In [621]:
def compute_loss(bag, preferred_order):
    strengths = np.array([bag.arguments[name].strength for name in preferred_order])
    i_idx, j_idx = np.triu_indices(len(strengths), k=1)
    sigma_diff = strengths[j_idx] - strengths[i_idx]
    return np.sum(np.maximum(0,sigma_diff))

In [622]:
# def compute_loss(bag, preferred_order):
#     strengths = np.array([bag.arguments[name].strength for name in preferred_order])
#     i_idx, j_idx = np.triu_indices(len(strengths), k=1)
#     sigma_diff = strengths[j_idx] - strengths[i_idx]
#     return np.sum(np.log1p(np.exp(sigma_diff)))

In [623]:
# compute gradient for the loss function
def compute_gradient(h, bag, preferred_order):
    gradient = {}
    original_base_scores = {arg.name: arg.initial_weight for arg in bag.arguments.values()}

    # original penalty
    grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)
    penalty = compute_loss(bag, preferred_order)

    for name, original_weight in original_base_scores.items():
        # perturb current argument
        for arg in bag.arguments.values():
            if arg.name == name:
                arg.reset_initial_weight(original_weight + h)
            else:
                arg.reset_initial_weight(original_base_scores[arg.name])

        grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)
        new_penalty = compute_loss(bag, preferred_order)
        gradient[name] = (new_penalty - penalty) / h

    # restore all base scores
    for arg in bag.arguments.values():
        arg.reset_initial_weight(original_base_scores[arg.name])
    grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

    return gradient

In [624]:
# adam optimiser
def adam_gradient(name, gradient, m, v, i):

    grad = gradient[name]
    # Adam optimiser parameters
    learning_rate = 0.1  # Initial learning rate
    beta1 = 0.85            # First-order moment decay rate
    beta2 = 0.98          # Second-order moment decay rate
    epsilon = 1e-8         # a small constant

    # update first-order moment and second-order moment
    m = beta1 * m + (1 - beta1) * grad
    v = beta2 * v + (1 - beta2) * (grad ** 2)

    # bias correction
    m_hat = m / (1 - beta1 ** i)
    v_hat = v / (1 - beta2 ** i)

    update = learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

    return update, m, v

In [625]:
# if the order is exactly the same as the desired order, then valid otherwise not.
def is_valid_order(bag, preferred_order):
    strengths = np.array([bag.arguments[name].strength for name in preferred_order])
    return np.all(strengths[:-1] >= strengths[1:])

In [626]:
# compute kendall and spearman correlations. if decreasing then 1, increasing 0.
def compute_kendall_spearman(predicted_strengths):
    if len(set(predicted_strengths)) == 1:
        return 1.0, 1.0

    ideal_descending = sorted(predicted_strengths, reverse=True)
    kendall_corr, _ = kendalltau(ideal_descending, predicted_strengths)
    spearman_corr, _ = spearmanr(ideal_descending, predicted_strengths)

    return kendall_corr, spearman_corr

In [627]:
# DF-QuAD gradual semantics
agg_f = grad.semantics.modular.ProductAggregation()
inf_f = grad.semantics.modular.LinearInfluence(conservativeness=1)

layer_sizes = [8,64,16,8,3]
N = 130
h = 10e-6
preferred_order = get_layer_nodes(layer_sizes, len(layer_sizes)-1)
input_args, hidden_args, output_args = get_layer_args(layer_sizes)
L1 = get_layer_nodes(layer_sizes, len(layer_sizes)-2) # last hidden layer

In [628]:
# # Bespoke:
immutable_args = L1

# #  input mutable
# immutable_args = hidden_args + output_args

# # hidden mutable
# immutable_args = input_args + output_args

# # input + hidden mutable
# immutable_args = output_args

# # all mutable
# immutable_args = []

In [629]:
# compute desired orderings for N MLP-like QBAFs
valid, kendall, spearman, time_total, base_score_diff = ([0] * N for _ in range(5))
mutable_num = sum(layer_sizes)-len(immutable_args) # number of mutable arguments
valid_itself = 0

for i in tqdm(range(N)):

    filename = f'../../bags/mlp_{i}.bag'
    bag = grad.BAG(filename)
    if is_valid_order(bag, preferred_order):
        valid_itself += 1
        valid[i] = 1
        kendall[i], spearman[i] = 1, 1
        continue

    start_time = time.time()
    m = {}
    v = {}
    original_base_scores = {arg.name: arg.initial_weight for arg in bag.arguments.values()}

    for iteration in range(1, 101):

        # compute gradient for all arguments
        gradient = compute_gradient(10e-6, bag, preferred_order)
        if all(value == 0 for value in gradient.values()): break

        # update Adam state and update base scores
        for arg in bag.arguments.values():
            if arg.name not in immutable_args:
                if arg.name not in m:
                    m[arg.name] = 0
                    v[arg.name] = 0

                current_weight = arg.get_initial_weight()
                adam_update, m[arg.name], v[arg.name] = adam_gradient(arg.name, gradient, m[arg.name], v[arg.name], iteration)
                new_weight = current_weight - adam_update
                new_weight = max(0, min(1, new_weight))
                arg.reset_initial_weight(new_weight)

        # recompute the strength and penalty
        grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

        if is_valid_order(bag, preferred_order):
            break



    if is_valid_order(bag, preferred_order):
        valid[i] = 1
        diff = 0
        for arg in bag.arguments.values():
            diff += abs(arg.initial_weight - original_base_scores[arg.name])
        if mutable_num:
            base_score_diff[i] = diff/mutable_num
    predicted_strengths = [bag.arguments[name].strength for name in preferred_order]
    kendall[i], spearman[i] = compute_kendall_spearman(predicted_strengths)
    time_total[i] = time.time()-start_time

print(f"valid_itself:{valid_itself}")
print(f"valid_avg:{(sum(valid)-valid_itself)/(N-valid_itself)}")
print(f"kendall_avg:{(sum(kendall)-valid_itself)/(N-valid_itself)}")
print(f"spearman_avg:{(sum(spearman)-valid_itself)/(N-valid_itself)}")
print(f"runtime_avg:{sum(time_total)/(N-valid_itself)}")
print(f"base_score_diff_avg:{sum(base_score_diff)/(sum(valid)-valid_itself)}")

100%|██████████| 130/130 [00:44<00:00,  2.94it/s]

valid_itself:30
valid_avg:1.0
kendall_avg:1.0
spearman_avg:1.0
runtime_avg:0.41653446197509764
base_score_diff_avg:0.006771756286176065
